# Frequently Asked Questions:

In [1]:
import tequila as tq

# Can I avoid re-translation/compilation on my objectives/circuits?

Yes you can. By calling `tq.compile` instead of `tq.simulate`.
This will give you back a callable objective.  
Check also the `basic usage` tutorial notebook

In [9]:
U = tq.gates.H(target=1) + tq.gates.Rx(angle="a", target=0, control=1)

# simulate the wavefunction with different variables
wfn0 = tq.simulate(U, variables={"a": 1.0})
wfn1 = tq.simulate(U, variables={"a": 2.0})

print(wfn0)
print(wfn1)

# the same, but avoiding re-compilation
# Note that your compiled object is translated to a quantum backend
# if the backend was not set, tequila it will pick the default which depends
# on which backends you have installed. You will seee it in the printout of the
# compiled circuits
compiled_U = tq.compile(U)
wfn0 = compiled_U(variables={"a":1.0})
wfn1 = compiled_U(variables={"a":2.0})

print("compiled circuit:", compiled_U)
print(wfn0)
print(wfn1)


# With Objectives it works in the same way
H = tq.paulis.Y(0)
E = tq.ExpectationValue(H=H, U=U)
objective = E**2 + 1.0

# simulate the objective with different variables
result0 = tq.simulate(objective, variables={"a": 1.0})
result1 = tq.simulate(objective, variables={"a": 2.0})

print("compiled objective:", objective)
print(result0)
print(result1)

# compile and then simulate
compiled_objective = tq.compile(objective)
result0 = compiled_objective(variables={"a":1.0})
result1 = compiled_objective(variables={"a":2.0})

print("compiled objective:", compiled_objective)
print(result0)
print(result1)

+0.7071|00> +0.6205|01> -0.3390i|11> 
+0.7071|00> +0.3821|01> -0.5950i|11> 
compiled circuit: <tequila.simulators.simulator_qulacs.BackendCircuitQulacs object at 0x7fdd70352be0>
+0.7071|00> +0.6205|01> -0.3390i|11> 
+0.7071|00> +0.3821|01> -0.5950i|11> 
compiled objective: Objective with 1(1) (unique) expectation values
Objective = f(E_0)
variables = [a]
1.1770184
1.2067055
compiled objective: Objective with 1(1) (unique) expectation values
Objective = f(E_0)
variables = [a]
1.1770184
1.2067055


# how can I run on a real quantum computer?
IBM's Qiskit can be used to run on some of IBM's public accessible quantum computers.
All you need for this is an ibm account (Follow the instructions under "Configure your IBM Quantum Experience credentials" here: https://github.com/Qiskit/qiskit-ibmq-provider).  
Tequila also supports Rigetti's PyQuil and Google's Cirq, but currently there are no publicly available devices.

Here is a small  example with Qiskit (you need to have qiskit installed for this):  
If you are already familiar with Qiskit and have special access rights you can also specific the provider by passing the keyword   
`qiskit_provider = <initialized_provider_instance>`   
Otherwise the default public provider will be initialized.  
Alternatively you can also externally initialize your backend and pass this down instead of a string

You always need to set samples if you intend to run on a real backend

I if you have access to Rigettis hardware you can use `backend="pyquil"` and pass down  
`pyquil_backend = dictonary`  
where the dictionary contains the instruction which go into pyquils `get_qc` function.

In [10]:
U = tq.gates.Ry(angle="a", target=0)
H = tq.paulis.X(0)
E = tq.ExpectationValue(H=H, U=U)

# simulate the square of the expectation value with a specific set of variables
result = tq.simulate(E**2, variables={"a":1.0}, samples=1000, backend="qiskit")

# run the same thing on one of IBM's quantum computers (check your ibm account for more information and keywords)
# note that the names of the computer might have changed
# here we use the 1-Qubit 'ibmq_armonk' 
result = tq.simulate(E**2, variables={"a":1.0}, samples=1000, backend="qiskit", qiskit_backend="ibmq_armonk") 

# In the SciPy Optimizers you can pass down all qiskit related options with the 
# `backend_options` key
# again you can also pass down a initialized qiskit backend object
backend_options = {"qiskit_backend":"ibmq_armonk"}
result = tq.optimizer_scipy.minimize(E**2, initial_values={"a":1.0}, samples=1000, backend="qiskit", backend_options=backend_options)

ObjectiveType is <class 'tequila.objective.objective.Objective'>
Starting BFGS optimization
Objective: 1 expectationvalues
Gradients: 3 expectationvalues (min=3, max=3)
Hessian: None

backend: <class 'tequila.simulators.simulator_qiskit.BackendExpectationValueQiskit'>
samples: 1000
1 active variables
E= 0.4489  angles= {a: 1.0}  samples= 1000
E= 0.11022399  angles= {a: 0.2940879464149475}  samples= 1000
E= 0.527076  angles= {a: -0.9714051337239711}  samples= 1000
E= 0.009604  angles= {a: 0.014596779994017095}  samples= 1000
E= 0.029584002  angles= {a: -0.23439378421529336}  samples= 1000
E= 0.0014439999  angles= {a: -0.06194057825106759}  samples= 1000
E= 0.00019600001  angles= {a: -0.10840037285555679}  samples= 1000
E= 0.024964003  angles= {a: -0.0830396865576113}  samples= 1000
E= 0.0081  angles= {a: -0.10793984869970413}  samples= 1000
E= 0.0025000002  angles= {a: -0.10839985851899529}  samples= 1000
E= 0.0017639999  angles= {a: -0.10840037285334388}  samples= 1000
E= 0.0067240004 

# Can I compile Objectives into different backends?
Yes you can. Tequila will print a warning if this happens. Warnings can be ignored by filtering them out (see the python warnings documentation)  

If a compiled circuit is used as input to compile then tequila will re-compile the circuit to the new backend (it it differs from the previous one)  

If a compiled objective is used as input to compile then tequila will only compile non-compiled expectationvalues into the different backend. Already compiled expectation values will remain untouched  

Note that you need at least two different backends for the following cell to execute.  
Just change the key to whatever you have installed.

In [4]:
backend1 = "qulacs"
backend2 = "cirq"

U = tq.gates.X(target=[0,1])
print("Example Circuit: ", U)
compiled_1 = tq.compile(U, backend=backend1)
compiled_2 = tq.compile(compiled_1, backend=backend2)
print("Circuit compiled to {} -> ".format(backend1), compiled_1)
print("Circuit compiled to {} -> ".format(backend1), compiled_1)

H = tq.paulis.X(0)*tq.paulis.Y(1) + tq.paulis.X([0,1])
print("\nmake objective with H = ", H)
objective = tq.ExpectationValue(H=H, U=U)
compiled_1 = tq.compile(objective, backend=backend1)

print("\nExpectationValues of objective 1:")
for expv in compiled_1.get_expectationvalues():
    print(expv)
    
objective2 = compiled_1 + objective # Its recommended to avoid those hybrids, but in principle it works

print("\nExpectationValues of partly compiled objective:")
for expv in objective2.get_expectationvalues():
    print(expv)
    
compiled_2 = tq.compile(objective2, backend=backend2)
print("\nExpectationValues of hybdrif compiled objective:")
for expv in compiled_2.get_expectationvalues():
    print(expv)
    





Example Circuit:  circuit: 
X(target=(0, 1), control=())

Circuit compiled to qulacs ->  <tequila.simulators.simulator_qulacs.BackendCircuitQulacs object at 0x7fc737bc0160>
Circuit compiled to qulacs ->  <tequila.simulators.simulator_qulacs.BackendCircuitQulacs object at 0x7fc737bc0160>

make objective with H =  +1.0000X(0)Y(1)+1.0000X(0)X(1)

ExpectationValues of objective 1:

ExpectationValues of partly compiled objective:

ExpectationValues of hybdrif compiled objective:


 /home/jsk/projects/OpenVQE/src/tequila/simulators/simulator_api.py:192: TequilaWarning:Looks like part the objective was already compiled for another backend.
Found ExpectationValue of type <class 'tequila.simulators.simulator_qulacs.BackendExpectationValueQulacs'> and <class 'tequila.simulators.simulator_cirq.BackendExpectationValueCirq'>
... proceeding with hybrid

 /home/jsk/projects/OpenVQE/src/tequila/simulators/simulator_api.py:192: TequilaWarning:Looks like part the objective was already compiled for another backend.
Found ExpectationValue of type <class 'tequila.objective.objective.ExpectationValueImpl'> and <class 'tequila.simulators.simulator_cirq.BackendExpectationValueCirq'>
... proceeding with hybrid

